In [ ]:
!pip install sentence_transformers
!pip install konlpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached sentence-transformers-2.2.2.tar.gz (85 kB)
  Using cached transformers-4.20.1-py3-none-any.whl (4.4 MB)
  Using cached sentencepiece-0.1.96-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.2 MB)
     |████████████████████████████████| 101 kB 7.1 MB/s 
     |████████████████████████████████| 596 kB 41.4 MB/s 
     |████████████████████████████████| 6.6 MB 20.5 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=bd9f93933fcd92eee757f63c2c965bca02eccbbf75750eb7a0521da3dfad9678
  Stored in directory: /root/.cache/pip/wheels/bf/06/fb/d59c1e5bd1dac7f6cf61ec0036cc3a10ab8fecaa6b2c3d3ee9
Successfully built sentence-transformers
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Looking in indexes: https://

In [ ]:
import numpy as np
import itertools

from konlpy.tag import Okt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer

In [ ]:
doc = """애플이 정부의 지원을 받는 해킹에서 저명 인사를 보호하기 위해 올가을 아이폰과 아이패드, 맥 컴퓨터 등에 '록다운 모드'를 도입한다고 밝혔다고 경제매체 CNBC가 6일(현지시간) 보도했다.
록다운 모드가 되면 아이폰에서 일부 기능이 꺼져 해커가 접근하거나 해킹할 수 있는 기능이 크게 줄면서 아이폰이 스파이웨어에 덜 취약해진다.
구체적으로는 아이메시지의 미리 보기 기능, 애플의 웹브라우저인 사파리의 자바스크립트 제한, 신규 설정 프로필 설치 차단, 유선 연결 차단, 화상통화인 페이스타임 등 수신형 서비스 요청 차단 등이다.
애플은 앞서 작년 9월 아이폰과 맥 컴퓨터 등에 중대한 보안 취약점이 있다며 긴급 소프트웨어 업데이트를 내놓은 바 있다.
이스라엘의 보안기업 NSO그룹이 만든 스파이웨어 '페가수스'를 이용하면 해커들이 클릭 한번 없이도 애플 기기를 감염시켜 카메라나 마이크를 켜고 검색 기록, 문자 메시지나 이메일 내용 등이 노출될 수 있다는 이유에서였다.
페가수스 같은 고도의 스파이웨어는 가격이 수천억원에 달해 평범한 해커가 아닌 정부나 경찰 기관 등이 주요 고객이다.
애플은 이 사건 뒤 미국 의회와 각국 정부에서 이 문제를 해결하라는 압력을 받아왔다.
록다운 모드는 국가의 지원을 받는 해커의 표적이 될 수 있어 극도로 고도의 보안 수준이 요구되는 소수의 이용자를 위한 것이다. 여기에는 고위 정치인이나 언론인, 인권 활동가, 기업 임원 등이 포함된다.
애플은 "대다수 이용자는 고도의 사이버 공격의 희생자가 될 일이 결코 없겠지만 희생자가 될 수 있는 소수의 사람을 보호하기 위해 쉬지 않고 일하겠다"고 밝혔다.
애플은 록다운 모드에 혹시 있을지 모를 보안상 허점을 발견하는 사람에게 최고 200만달러(약 26억원)를 지급하겠다며 '현상금'까지 내걸었다."""

In [ ]:
#형태소 분석기를 통해 명사만 추출할 문서 만들기
okt = Okt()

tokenized_doc = okt.pos(doc)
tokenized_nouns = ' '.join([word[0] for word in tokenized_doc if word[1] == 'Noun'])
print('품사 태깅 10개만 출력 :', tokenized_doc[:10])
print('명사 추출 :', tokenized_nouns)

품사 태깅 10개만 출력 : [('애플', 'Noun'), ('이', 'Josa'), ('정부', 'Noun'), ('의', 'Josa'), ('지원', 'Noun'), ('을', 'Josa'), ('받는', 'Verb'), ('해킹', 'Noun'), ('에서', 'Josa'), ('저명', 'Noun')]
명사 추출 : 애플 정부 지원 해킹 저명 인사 보호 위해 올가을 아이폰 아이패드 맥 컴퓨터 등 록 다운 모드 를 도입 경제 매체 현지 시간 보도 록 다운 모드 아이폰 일부 기능 해커 접근 거나 해킹 수 기능 크게 아이폰 스파이웨어 덜 취약 구체 아이메시지 미리 보기 기능 애플 웹브라우저 사파리 자바스크립트 제한 신규 설정 프로필 설치 차단 유선 연결 차단 화상통화 페이스타임 등 신형 서비스 요청 차단 등 애플 앞서 작년 아이폰 맥 컴퓨터 등 보안 취약점 긴급 소프트웨어 업데이트 바 이스라엘 보안 기업 그룹 스파이웨어 페가수스 를 이용 해커 클릭 한번 애플 기기 감염 카메라 마이크 검색 기록 문자 메시지 이메일 내용 등 노출 수 이유 페가수스 고도 스파이웨어 가격 억원 해커 정부 경찰 기관 등 주요 고객 애플 이 사건 뒤 미국 의회 각국 정부 이 문제 해결 압력 록 다운 모드 국가 지원 해커 표적 수 극도 고도 보안 수준 요구 소수 이용자 위 것 여기 고위 정치인 언론인 인권 활동가 기업 임원 등 포함 애플 대다수 이용자 고도 사이버 공격 희생 결코 희생 수 소수 사람 보호 위해 쉬 일 고 애플 록 다운 모드 혹시 보안 점 발견 사람 최고 약 를 지급 현상금 내


In [ ]:
#여기서는 사이킷런의 CountVectorizer를 사용하여 단어 추출 (n-gram 추출 쉬움)

n_gram_range = (2,3)

count = CountVectorizer(ngram_range=n_gram_range).fit([tokenized_nouns])
candidates = count.get_feature_names_out()

print('gram 개수 :', len(candidates))
print('gram 다섯개만 출력 :', candidates[:5])

gram 개수 : 305
gram 다섯개만 출력 : ['가격 억원' '가격 억원 해커' '각국 정부' '각국 정부 문제' '감염 카메라']


In [ ]:
#이제 문서와 문서로부터 추출한 키워드들을 SBERT를 통해서 수치화한다
model = SentenceTransformer('sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens')

In [ ]:
doc_embedding = model.encode([doc])
candidate_embeddings = model.encode(candidates)

In [ ]:
#이제 문서와 가장 유사한 키워드 추출.(문서를 대표하기 위한 좋은 키워드)
top_n = 10
distances = cosine_similarity(doc_embedding, candidate_embeddings)
keywords = [candidates[index] for index in distances.argsort()[0][-top_n:]]
print(keywords)

['고객 애플 사건', '일부 기능 해커', '주요 고객 애플', '애플 웹브라우저 사파리', '애플 기기 감염', '애플 웹브라우저', '보호 위해 애플', '애플 정부 지원', '아이폰 스파이웨어 취약', '아이폰 컴퓨터 보안']


In [ ]:
distances

array([[-3.86920199e-02,  3.76439542e-01, -1.21156350e-01,
        -4.13071141e-02,  9.65741351e-02,  9.71842557e-02,
         1.79692477e-01,  3.15241158e-01,  2.23368928e-02,
        -1.00282393e-02, -7.56385028e-02,  9.60360467e-03,
         1.24056019e-01,  1.05430469e-01,  6.96671903e-02,
         2.72480175e-02,  4.19590116e-01,  4.52743232e-01,
         1.56958580e-01,  1.74651057e-01,  2.24895537e-01,
         2.73111343e-01,  1.85443982e-01,  2.67823428e-01,
        -6.40758965e-03,  6.31761104e-02, -6.76746294e-02,
        -8.24632794e-02, -8.46472755e-02, -2.43162960e-02,
         2.97456253e-02,  4.23026979e-01,  1.32193893e-01,
         3.87137569e-02, -1.00299016e-01,  1.26885504e-01,
        -9.94826108e-02,  8.43395144e-02,  5.36460727e-02,
         2.20051467e-01,  1.61720276e-01,  3.86850178e-01,
        -6.07943386e-02,  3.22048962e-01,  4.03184444e-01,
         4.34595972e-01, -8.34488347e-02, -4.17380407e-03,
        -4.24735062e-02,  2.62084097e-01,  4.33910415e-0

# Max Sum Similarity

후보 간의 유사성 최소화, 문서와의 후보 유사성 극대화

In [ ]:
def max_sum_sim(doc_embedding, candidate_embeddings, words, top_n, nr_candidates):
  #문서와 각 키워드들 간의 유사도
  distances = cosine_similarity(doc_embedding, candidate_embeddings)
  #각 키워드들 간의 유사도
  distances_candidates = cosine_similarity(candidate_embeddings, candidate_embeddings)
  #코사인 유사도에 기반하여 키워드들 중 상위 top_n개의 단어를 pick
  words_idx = list(distances.argsort()[0][-nr_candidates:])
  words_vals = [candidates[index] for index in words_idx]
  distances_candidates = distances_candidates[np.ix_(words_idx, words_idx)]

  #각 키워드들 중에서 가장 덜 유사한 키워드들간의 조합을 계산
  min_sim = np.inf
  candidate = None
  for combination in itertools.combinations(range(len(words_idx)), top_n):
    sim = sum([distances_candidates[i][j] for i in combination for j in combination if i != j])
    if sim < min_sim:
      candidate = combination
      min_sim = sim

  return [words_vals[idx] for idx in candidate]

In [ ]:
#상위 10개 키워드 중에 서로 가장 유사성 낮은 5개 선택
max_sum_sim(doc_embedding, candidate_embeddings, candidates, top_n=5, nr_candidates=10)

['일부 기능 해커', '애플 웹브라우저', '보호 위해 애플', '애플 정부 지원', '아이폰 스파이웨어 취약']

In [ ]:
max_sum_sim(doc_embedding, candidate_embeddings, candidates, top_n=5, nr_candidates=20) #높으면 더 다양

['아이폰 아이패드 컴퓨터', '해커 정부 경찰', '컴퓨터 보안 취약점', '애플 앞서 작년', '보호 위해 애플']

# Maximal Marginal Relevance

텍스트 요약 작업에서 중복 최소화, 결과 다양성 극대화

문서와 가장 유사한 키워드/키프레이즈 선택. 문서와 유사하고 이미 선택된 키워드/키프레이즈와 유사하지 않은 새로운 후보 반복적 선택

In [ ]:
def mmr(doc_embedding, candidate_embeddings, words, top_n, diversity):
  #문서와 각 키워드들 간의 유사도가 적힌 리스트
  word_doc_similarity = cosine_similarity(candidate_embeddings, doc_embedding)
  #각 키워드들 간의 유사도
  word_similarity = cosine_similarity(candidate_embeddings, candidate_embeddings)
  #문서와 가장 높은 유사도를 가진 키워드의 인덱스 추출
  keywords_idx = [np.argmax(word_doc_similarity)]
  #가장 높은 유사도를 가진 키워드의 인덱스를 제외한 문서의 인덱스들
  candidates_idx = [i for i in range(len(words)) if i != keywords_idx[0]]
  #최고의 키워드는 이미 추출. top_n-1번만큼 아래를 반복
  for _ in range(top_n-1):
    candidate_similarities = word_doc_similarity[candidates_idx, :]
    target_similarities = np.max(word_similarity[candidates_idx][:, keywords_idx], axis=1)

    #MMR 계산
    mmr = (1-diversity)*candidate_similarities - diversity*target_similarities.reshape(-1,1)
    mmr_idx = candidates_idx[np.argmax(mmr)]

    #keywords & candidates 업데이트
    keywords_idx.append(mmr_idx)
    candidates_idx.remove(mmr_idx)

  return [words[idx] for idx in keywords_idx]

In [ ]:
mmr(doc_embedding, candidate_embeddings, candidates, top_n=5, diversity=0.2)

['아이폰 컴퓨터 보안', '애플 정부 지원', '일부 기능 해커', '아이폰 스파이웨어 취약', '보호 위해 애플']

In [ ]:
mmr(doc_embedding, candidate_embeddings, candidates, top_n=5, diversity=0.7)  #높으면 더 다양

['아이폰 컴퓨터 보안', '각국 정부 문제', '정치인 언론인 인권', '기기 감염', '기업 임원']

# 애플 다른 주제로 키워드 뽑아보기

In [ ]:
doc = """인천시 남동구 서창2동 행정복지센터가 애플어린이집으로부터 취약계층을 위한 후원금 150만 원을 전달받았다고 6일 밝혔다.
후원금은 지난 5월 어린이집에서 원생을 비롯한 학부모, 교직원, 지역주민이 뜻을 모아 진행한 플리마켓(벼룩시장) 수익금으로, 서창2동 취약계층을 위한 복지기금으로 전액 기탁했다.
하현주 애플어린이집 원장은“기부를 통해 아이들이 소외된 이웃과 따뜻한 마음을 나누는 기쁨을 경험하는 소중한 기회가 됐다"며 우리 동 어려운 이웃을 위해 도움을 보태고자 하반기에도 기부를 추진할 예정이라고 전했다.
박미경 서창2동장은“소중한 나눔을 실천해주신 애플어린이집에 감사드리며, 전달받은 후원금은 아동이 많이 거주하는 우리 동 특성에 맞춰 아이들을 위한 복지사업 추진을 위해 가치 있게 쓰겠다.”라고 말했다."""

In [ ]:
#형태소 분석기를 통해 명사만 추출할 문서 만들기
okt = Okt()

tokenized_doc = okt.pos(doc)
tokenized_nouns = ' '.join([word[0] for word in tokenized_doc if word[1] == 'Noun'])
print('품사 태깅 10개만 출력 :', tokenized_doc[:10])
print('명사 추출 :', tokenized_nouns)

품사 태깅 10개만 출력 : [('인천', 'Noun'), ('시', 'Noun'), ('남동구', 'Noun'), ('서창', 'Noun'), ('2', 'Number'), ('동', 'Modifier'), ('행정', 'Noun'), ('복지', 'Noun'), ('센터', 'Noun'), ('가', 'Josa')]
명사 추출 : 인천 시 남동구 서창 행정 복지 센터 애플 어린이집 취약 계층 위 후 원금 원 전달 후원 금은 지난 어린이집 생 비롯 학부모 교직원 지역 주민 뜻 진행 플리 마켓 벼룩시장 수익금 서창 취약 계층 위 복지 기금 전액 기탁 하현 주 애플 어린이집 원장 기부 통해 아이 소외 이웃 마음 기쁨 경험 기회 며 우리 이웃 위해 도움 보태 고자 하반기 기부 추진 예정 전 박미경 서창 동장 나눔 실천 애플 어린이집 감사 전달 후원 금은 아동 거주 우리 특성 아이 위 복지 사업 추진 위해 가치 말


In [ ]:
#여기서는 사이킷런의 CountVectorizer를 사용하여 단어 추출 (n-gram 추출 쉬움)

n_gram_range = (2,3)

count = CountVectorizer(ngram_range=n_gram_range).fit([tokenized_nouns])
candidates = count.get_feature_names_out()

print('gram 개수 :', len(candidates))
print('gram 다섯개만 출력 :', candidates[:5])

gram 개수 : 145
gram 다섯개만 출력 : ['감사 전달' '감사 전달 후원' '거주 우리' '거주 우리 특성' '경험 기회']


In [ ]:
#이제 문서와 문서로부터 추출한 키워드들을 SBERT를 통해서 수치화한다
model = SentenceTransformer('sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens')

In [ ]:
doc_embedding = model.encode([doc])
candidate_embeddings = model.encode(candidates)

In [ ]:
#이제 문서와 가장 유사한 키워드 추출.(문서를 대표하기 위한 좋은 키워드)
top_n = 5
distances = cosine_similarity(doc_embedding, candidate_embeddings)
keywords = [candidates[index] for index in distances.argsort()[0][-top_n:]]
print(keywords)

['후원 금은 아동', '아이 복지 사업', '어린이집 감사 전달', '애플 어린이집 감사', '기부 통해 아이']


# Max Sum Similarity

후보 간의 유사성 최소화, 문서와의 후보 유사성 극대화

In [ ]:
def max_sum_sim(doc_embedding, candidate_embeddings, words, top_n, nr_candidates):
  #문서와 각 키워드들 간의 유사도
  distances = cosine_similarity(doc_embedding, candidate_embeddings)
  #각 키워드들 간의 유사도
  distances_candidates = cosine_similarity(candidate_embeddings, candidate_embeddings)
  #코사인 유사도에 기반하여 키워드들 중 상위 top_n개의 단어를 pick
  words_idx = list(distances.argsort()[0][-nr_candidates:])
  words_vals = [candidates[index] for index in words_idx]
  distances_candidates = distances_candidates[np.ix_(words_idx, words_idx)]

  #각 키워드들 중에서 가장 덜 유사한 키워드들간의 조합을 계산
  min_sim = np.inf
  candidate = None
  for combination in itertools.combinations(range(len(words_idx)), top_n):
    sim = sum([distances_candidates[i][j] for i in combination for j in combination if i != j])
    if sim < min_sim:
      candidate = combination
      min_sim = sim

  return [words_vals[idx] for idx in candidate]

In [ ]:
#상위 10개 키워드 중에 서로 가장 유사성 낮은 5개 선택
max_sum_sim(doc_embedding, candidate_embeddings, candidates, top_n=5, nr_candidates=10)

['이웃 마음 기쁨', '하현 애플 어린이집', '실천 애플 어린이집', '어린이집 감사', '아이 복지 사업']

In [ ]:
max_sum_sim(doc_embedding, candidate_embeddings, candidates, top_n=5, nr_candidates=20) #높으면 더 다양

['애플 어린이집 취약', '계층 복지 기금', '어린이집 비롯 학부모', '나눔 실천 애플', '이웃 마음 기쁨']

# Maximal Marginal Relevance

텍스트 요약 작업에서 중복 최소화, 결과 다양성 극대화

문서와 가장 유사한 키워드/키프레이즈 선택. 문서와 유사하고 이미 선택된 키워드/키프레이즈와 유사하지 않은 새로운 후보 반복적 선택

In [ ]:
def mmr(doc_embedding, candidate_embeddings, words, top_n, diversity):
  #문서와 각 키워드들 간의 유사도가 적힌 리스트
  word_doc_similarity = cosine_similarity(candidate_embeddings, doc_embedding)
  #각 키워드들 간의 유사도
  word_similarity = cosine_similarity(candidate_embeddings, candidate_embeddings)
  #문서와 가장 높은 유사도를 가진 키워드의 인덱스 추출
  keywords_idx = [np.argmax(word_doc_similarity)]
  #가장 높은 유사도를 가진 키워드의 인덱스를 제외한 문서의 인덱스들
  candidates_idx = [i for i in range(len(words)) if i != keywords_idx[0]]
  #최고의 키워드는 이미 추출. top_n-1번만큼 아래를 반복
  for _ in range(top_n-1):
    candidate_similarities = word_doc_similarity[candidates_idx, :]
    target_similarities = np.max(word_similarity[candidates_idx][:, keywords_idx], axis=1)

    #MMR 계산
    mmr = (1-diversity)*candidate_similarities - diversity*target_similarities.reshape(-1,1)
    mmr_idx = candidates_idx[np.argmax(mmr)]

    #keywords & candidates 업데이트
    keywords_idx.append(mmr_idx)
    candidates_idx.remove(mmr_idx)

  return [words[idx] for idx in keywords_idx]

In [ ]:
mmr(doc_embedding, candidate_embeddings, candidates, top_n=5, diversity=0.2)

['기부 통해 아이', '애플 어린이집 감사', '어린이집 감사 전달', '아이 복지 사업', '후원 금은 아동']

In [ ]:
mmr(doc_embedding, candidate_embeddings, candidates, top_n=5, diversity=0.7)  #높으면 더 다양

['기부 통해 아이', '인천 남동구 서창', '기탁 하현 애플', '사업 추진', '소외 이웃 마음']

# 함수 설정

In [ ]:
def candidates_fn(doc, n_gram_range):
  okt = Okt()

  tokenized_doc = okt.pos(doc)
  tokenized_nouns = ' '.join([word[0] for word in tokenized_doc if word[1] == 'Noun'])

  count = CountVectorizer(ngram_range=n_gram_range).fit([tokenized_nouns])
  candidates = count.get_feature_names_out()

  return candidates

In [ ]:
def embedding_fn(doc, candidates):
  #이제 문서와 문서로부터 추출한 키워드들을 SBERT를 통해서 수치화한다
  model = SentenceTransformer('sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens')

  doc_embedding = model.encode([doc])
  candidate_embeddings = model.encode(candidates)

  return (doc_embedding, candidate_embeddings)

In [ ]:
def keywords_extract(doc_embedding, candidate_embeddings, candidates, top_n):
  distances = cosine_similarity(doc_embedding, candidate_embeddings)
  keywords = [candidates[index] for index in distances.argsort()[0][-top_n:]]

  return keywords

In [ ]:
def max_sum_sim(doc_embedding, candidate_embeddings, candidates, top_n, nr_candidates):
  #문서와 각 키워드들 간의 유사도
  distances = cosine_similarity(doc_embedding, candidate_embeddings)
  #각 키워드들 간의 유사도
  distances_candidates = cosine_similarity(candidate_embeddings, candidate_embeddings)
  #코사인 유사도에 기반하여 키워드들 중 상위 top_n개의 단어를 pick
  words_idx = list(distances.argsort()[0][-nr_candidates:])
  words_vals = [candidates[index] for index in words_idx]
  distances_candidates = distances_candidates[np.ix_(words_idx, words_idx)]

  #각 키워드들 중에서 가장 덜 유사한 키워드들간의 조합을 계산
  min_sim = np.inf
  candidate = None
  for combination in itertools.combinations(range(len(words_idx)), top_n):
    sim = sum([distances_candidates[i][j] for i in combination for j in combination if i != j])
    if sim < min_sim:
      candidate = combination
      min_sim = sim

  return [words_vals[idx] for idx in candidate]

In [ ]:
def mmr(doc_embedding, candidate_embeddings, words, top_n, diversity):
  #문서와 각 키워드들 간의 유사도가 적힌 리스트
  word_doc_similarity = cosine_similarity(candidate_embeddings, doc_embedding)
  #각 키워드들 간의 유사도
  word_similarity = cosine_similarity(candidate_embeddings, candidate_embeddings)
  #문서와 가장 높은 유사도를 가진 키워드의 인덱스 추출
  keywords_idx = [np.argmax(word_doc_similarity)]
  #가장 높은 유사도를 가진 키워드의 인덱스를 제외한 문서의 인덱스들
  candidates_idx = [i for i in range(len(words)) if i != keywords_idx[0]]
  #최고의 키워드는 이미 추출. top_n-1번만큼 아래를 반복
  for _ in range(top_n-1):
    candidate_similarities = word_doc_similarity[candidates_idx, :]
    target_similarities = np.max(word_similarity[candidates_idx][:/, keywords_idx], axis=1)

    #MMR 계산
    mmr = (1-diversity)*candidate_similarities - diversity*target_similarities.reshape(-1,1)
    mmr_idx = candidates_idx[np.argmax(mmr)]

    #keywords & candidates 업데이트
    keywords_idx.append(mmr_idx)
    candidates_idx.remove(mmr_idx)

  return [words[idx] for idx in keywords_idx]

In [ ]:
candidates = candidates_fn(doc, (2,3))

In [ ]:
doc_embedding, candidate_embeddings = embedding_fn(doc, candidates)

In [ ]:
keywords = keywords_extract(doc_embedding, candidate_embeddings, candidates, 5)
keywords

In [ ]:
mss_keywords = max_sum_sim(doc_embedding, candidate_embeddings, candidates, top_n=5, nr_candidates=20) #높으면 더 다양
mss_keywords

In [ ]:
mmr_keywords = mmr(doc_embedding, candidate_embeddings, candidates, top_n=5, diversity=0.7)  #높으면 더 다양
mmr_keywords